In [1]:
import rospy
import numpy as np
import sys

#import open3d as o3d
from scipy import spatial
from scipy.spatial.transform import Rotation as R
import math
import tf2_ros, tf
import geometry_msgs.msg
import PyKDL
import time
import moveit_commander
import moveit_msgs.msg
from moveit_commander.conversions import pose_to_list

image = None

moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node("my_motion", anonymous=True)

robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()
group_name = "panda_arm"
move_group = moveit_commander.MoveGroupCommander(group_name) #we'll pass it on while calling functions




tfbuffer = tf2_ros.Buffer()
listener = tf2_ros.TransformListener(tfbuffer)
br = tf2_ros.TransformBroadcaster()
t = geometry_msgs.msg.TransformStamped()
static_br = tf2_ros.StaticTransformBroadcaster()
static_t = geometry_msgs.msg.TransformStamped()

loop_rate = rospy.Rate(0.5) # Node cycle rate (in Hz).
np.set_printoptions(suppress=True)


## Create a `DisplayTrajectory`_ ROS publisher which is used to display
## trajectories in Rviz:
display_trajectory_publisher = rospy.Publisher("/move_group/display_planned_path", moveit_msgs.msg.DisplayTrajectory, queue_size=20,)


print("============ Printing robot current Joint values") #O/P: current joint values of the end-effector
#print(move_group.get_current_joint_values())
print("In Degrees: ", np.degrees( move_group.get_current_joint_values() ) ) #O/P joint angles in degrees
print("")

print("============ Printing robot current pose") #O/P will be XYZ and xyz w  positions and orientations of the robot.
print(move_group.get_current_pose())
print("")

print("============ Printing robot current RPY") #O/P wil be orientation of end-effector in RPY (Radians)
#print(move_group.get_current_rpy())
print("RPY In Degrees: ", np.degrees( move_group.get_current_rpy() ) )
print("")
z_offset = 0.3

============ Printing robot current Joint values
In Degrees:  [  -0.01056111  -44.85081501    0.00934041 -135.23826566   -0.01893858
   89.85587847   45.00280127]

============ Printing robot current pose
header: 
  seq: 0
  stamp: 
    secs: 2425
    nsecs:   1000000
  frame_id: "world"
pose: 
  position: 
    x: 0.3071791097877493
    y: 7.302316087318123e-05
    z: 0.5873017719057154
  orientation: 
    x: 0.923909386683293
    y: -0.382583108972439
    z: -0.004204011808465989
    w: 0.00198398896125949

============ Printing robot current RPY
RPY In Degrees:  [179.60548028   0.35787523 -44.98820113]



In [29]:
def go_to_coord_goal(move_group,coord):
    pose_goal = geometry_msgs.msg.Pose()
    pose_goal.position.x = coord[0]
    pose_goal.position.y = coord[1]
    pose_goal.position.z = coord[2]
    pose_goal.orientation.x = coord[3]
    pose_goal.orientation.y = coord[4]
    pose_goal.orientation.z = coord[5]
    pose_goal.orientation.w = coord[6]

    move_group.set_pose_target(pose_goal)
    
    success = move_group.go(wait=True)
    
    move_group.stop()
    move_group.clear_pose_targets()
    current_pose = move_group.get_current_pose().pose



def fetch_transform(tfbuffer,frame1,frame2,quat=0):
    flag = 0
    while flag==0:
        try:
            trans = tfbuffer.lookup_transform(frame1, frame2, rospy.Time(),rospy.Duration(8.0))
            #print (trans)
            trans = trans.transform  #save translation and rotation
            #rot = PyKDL.Rotation.Quaternion(* [ trans.rotation.x,trans.rotation.y,trans.rotation.z,trans.rotation.w] )
            #ypr = [ i  / np.pi * 180 for i in rot.GetEulerZYX() ]
            #print(ypr[2],ypr[1],ypr[0])
            
            rot = R.from_quat([ trans.rotation.x,trans.rotation.y,trans.rotation.z,trans.rotation.w]) #creates rotation matrix
            rpy = rot.as_euler('XYZ',degrees=True) #extrinsic
            break
        except (tf2_ros.LookupException, tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException) as e:
            #print ("Fail", e)
            continue
    if quat==0:
        return trans.translation.x, trans.translation.y, trans.translation.z, rpy[0],rpy[1],rpy[2] #ypr[2], ypr[1], ypr[0]
    elif quat==1:
        return trans.translation.x, trans.translation.y, trans.translation.z, trans.rotation.x, trans.rotation.y, trans.rotation.z, trans.rotation.w
    

def Publish_coordinates(Coordinates, parent_name, child_name, static = False): #Coordinates expects [[0,0,0,0,0,0,0],[1,1,1,1,1,1,1]] format

    for index in range (len(Coordinates)):
        
        if static==True:
            static_t.header.stamp = rospy.Time.now()
            static_t.header.frame_id = parent_name
            static_t.child_frame_id = child_name + "_static_"+str(index)
            static_t.transform.translation.x = Coordinates[index][0]
            static_t.transform.translation.y = Coordinates[index][1]
            static_t.transform.translation.z = Coordinates[index][2]

            #r_quat = tf.transformations.quaternion_from_euler(Roll,Pitch,Yaw)
            static_t.transform.rotation.x = Coordinates[index][3]
            static_t.transform.rotation.y = Coordinates[index][4]
            static_t.transform.rotation.z = Coordinates[index][5]
            static_t.transform.rotation.w = Coordinates[index][6]
            static_br.sendTransform(static_t)
        else:
            t.header.frame_id = parent_name
            t.child_frame_id = child_name + "_"+str(index)

            t.header.stamp = rospy.Time.now()
            t.transform.translation.x = Coordinates[index][0]
            t.transform.translation.y = Coordinates[index][1]
            t.transform.translation.z = Coordinates[index][2] 

            #r_quat = tf.transformations.quaternion_from_euler(Roll,Pitch,Yaw)
            t.transform.rotation.x = Coordinates[index][3]
            t.transform.rotation.y = Coordinates[index][4]
            t.transform.rotation.z = Coordinates[index][5]
            t.transform.rotation.w = Coordinates[index][6]
            br.sendTransform(t)


### This code will publish Camera target, where depth cam is supposed to go, visualize in Rviz

In [11]:
transform_world_plane = fetch_transform(tfbuffer,'world', 'plane_static_0',quat=1)
transform_link8_camera_depth_optical_frame = fetch_transform(tfbuffer,'panda_link8',
                                                             'camera_depth_optical_frame',quat=1)

#Translations
trans_x =  transform_world_plane[0]
trans_y = transform_world_plane[1]
trans_z = transform_world_plane[2]

KDL_original_frame = PyKDL.Frame(PyKDL.Rotation.Quaternion(transform_world_plane[3],transform_world_plane[4], 
                                           transform_world_plane[5], transform_world_plane[6]),
                 PyKDL.Vector(trans_x, trans_y, trans_z))

trans_x,trans_y,trans_z = KDL_original_frame * PyKDL.Vector(0, 0, z_offset) #Add offset

#ROTATIONS
#link8_to_cam_frame = R.from_quat([transform_link8_camera_depth_optical_frame[3], 
#                                  transform_link8_camera_depth_optical_frame[4], 
#                                  transform_link8_camera_depth_optical_frame[5], 
#                                  transform_link8_camera_depth_optical_frame[6]]) 

original_frame = R.from_quat([transform_world_plane[3], 
                              transform_world_plane[4], 
                              transform_world_plane[5], 
                              transform_world_plane[6]])

flip_frame = R.from_euler('xyz', [180., 0., 180.], degrees=True) #To invert the frame

final_frame = original_frame * flip_frame #link8_to_cam_frame * original_frame * flip_frame
r_quat = final_frame.as_quat()

final_coordinates = [trans_x, trans_y, trans_z, r_quat[0], r_quat[1], r_quat[2], r_quat[3]]
Publish_coordinates([final_coordinates], "world", 'Camera_Target', static = True)


### This code does all the frame multiplication to make sure eef goes to its own target offset by camera target.

In [12]:
#EEF
#Translations
trans_x = transform_world_plane[0]
trans_y = transform_world_plane[1]
trans_z = transform_world_plane[2]

KDL_original_frame = PyKDL.Frame(PyKDL.Rotation.Quaternion(transform_world_plane[3],transform_world_plane[4], 
                                           transform_world_plane[5], transform_world_plane[6]),
                 PyKDL.Vector(trans_x, trans_y, trans_z))

trans_x,trans_y,trans_z = KDL_original_frame * PyKDL.Vector(transform_link8_camera_depth_optical_frame[0]
                                            ,transform_link8_camera_depth_optical_frame[1]
                                            ,transform_link8_camera_depth_optical_frame[2]+z_offset) # #Add offset

#ROTATIONS
link8_to_cam_frame = R.from_quat([transform_link8_camera_depth_optical_frame[3], 
                                  transform_link8_camera_depth_optical_frame[4], 
                                  transform_link8_camera_depth_optical_frame[5], 
                                  transform_link8_camera_depth_optical_frame[6]]) 

original_frame = R.from_quat([transform_world_plane[3], transform_world_plane[4], 
                              transform_world_plane[5], transform_world_plane[6]])

flip_frame = R.from_euler('xyz', [180., 0., 180.], degrees=True) #To invert the frame

final_frame = original_frame * link8_to_cam_frame * flip_frame 
r_quat = final_frame.as_quat()

final_coordinates = [trans_x, trans_y, trans_z, r_quat[0], r_quat[1], r_quat[2], r_quat[3]]
Publish_coordinates([final_coordinates], "world", 'EEF_Target', static = False)


### This code moves the robot EEF to desired coordinates

In [13]:
#time.sleep(2)
go_to_coord_goal(move_group, final_coordinates) #pass values to function to make robot move in cartesian space. 
print("Moving to Target position")
print("")
#time.sleep(2)


Moving to Target position



In [27]:
transform_world_plane = fetch_transform(tfbuffer,'world', 'plane_static_0',quat=1)
transform_link8_camera_depth_optical_frame = fetch_transform(tfbuffer,'panda_link8',
                                                             'camera_depth_optical_frame',quat=1)

#Translations
trans_x =  transform_world_plane[0]
trans_y = transform_world_plane[1]
trans_z = transform_world_plane[2]

KDL_original_frame = PyKDL.Frame(PyKDL.Rotation.Quaternion(transform_world_plane[3],transform_world_plane[4], 
                                           transform_world_plane[5], transform_world_plane[6]),
                 PyKDL.Vector(trans_x, trans_y, trans_z))

trans_x,trans_y,trans_z = KDL_original_frame * PyKDL.Vector(0, 0, z_offset) #Add offset

#ROTATIONS
#link8_to_cam_frame = R.from_quat([transform_link8_camera_depth_optical_frame[3], 
#                                  transform_link8_camera_depth_optical_frame[4], 
#                                  transform_link8_camera_depth_optical_frame[5], 
#                                  transform_link8_camera_depth_optical_frame[6]]) 

original_frame = R.from_quat([transform_world_plane[3], 
                              transform_world_plane[4], 
                              transform_world_plane[5], 
                              transform_world_plane[6]])

flip_frame = R.from_euler('xyz', [180., 0., 180.], degrees=True) #To invert the frame

final_frame = original_frame * flip_frame #link8_to_cam_frame * original_frame * flip_frame
r_quat = final_frame.as_quat()

final_coordinates = [trans_x, trans_y, trans_z, r_quat[0], r_quat[1], r_quat[2], r_quat[3]]
Publish_coordinates([final_coordinates], "world", 'Camera_Target', static = True)




#EEF
#Translations
trans_x = transform_world_plane[0]
trans_y = transform_world_plane[1]
trans_z = transform_world_plane[2]

KDL_original_frame = PyKDL.Frame(PyKDL.Rotation.Quaternion(transform_world_plane[3],transform_world_plane[4], 
                                           transform_world_plane[5], transform_world_plane[6]),
                 PyKDL.Vector(trans_x, trans_y, trans_z))

trans_x,trans_y,trans_z = KDL_original_frame * PyKDL.Vector(transform_link8_camera_depth_optical_frame[0]
                                            ,transform_link8_camera_depth_optical_frame[1]
                                            ,transform_link8_camera_depth_optical_frame[2]+z_offset) # #Add offset

#ROTATIONS
link8_to_cam_frame = R.from_quat([transform_link8_camera_depth_optical_frame[3], 
                                  transform_link8_camera_depth_optical_frame[4], 
                                  transform_link8_camera_depth_optical_frame[5], 
                                  transform_link8_camera_depth_optical_frame[6]]) 

original_frame = R.from_quat([transform_world_plane[3], transform_world_plane[4], 
                              transform_world_plane[5], transform_world_plane[6]])

flip_frame = R.from_euler('xyz', [180., 0., 180.], degrees=True) #To invert the frame

final_frame = original_frame * link8_to_cam_frame * flip_frame 
r_quat = final_frame.as_quat()

final_coordinates = [trans_x, trans_y, trans_z, r_quat[0], r_quat[1], r_quat[2], r_quat[3]]
Publish_coordinates([final_coordinates], "world", 'EEF_Target', static = False)


time.sleep(2)
go_to_coord_goal(move_group, final_coordinates) #pass values to function to make robot move in cartesian space. 
print("Moving to Target position")
print("")


Moving to Target position

